In [1]:
import os
import sys
for path in sys.path:
    print(path)

sys.path.append('..')

print(os.getcwd())

confidential_phenopackets_path = ''

C:\Users\Surface\OneDrive\Documents\DataSpell\ERKER2Phenopackets\ERKER2Phenopackets\src\analysis\ml
C:\Users\Surface\OneDrive\Documents\DataSpell\ERKER2Phenopackets
C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets\python310.zip
C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets\DLLs
C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets\lib
C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets

C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets\lib\site-packages
C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets\lib\site-packages\win32
C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets\lib\site-packages\win32\lib
C:\Users\Surface\anaconda3\envs\ERKER2Phenopackets\lib\site-packages\Pythonwin
C:\Users\Surface\OneDrive\Documents\DataSpell\ERKER2Phenopackets\ERKER2Phenopackets\src\analysis\ml


In [2]:
import configparser
from pathlib import Path

config = configparser.ConfigParser()
config.read('../../../data/config/config.cfg')

phenopackets_out_dir = ('..' / Path(config.get('Paths', 'phenopackets_out'))).resolve()

phenopackets_out_dir

WindowsPath('C:/Users/Surface/OneDrive/Documents/DataSpell/ERKER2Phenopackets/ERKER2Phenopackets/data/out/phenopackets')

# read in data from phenopackets

In [3]:
from ERKER2Phenopackets.src.utils.io import read_files

example_phenopackets_dir = phenopackets_out_dir / 'example-phenopackets-from-synthetic-data'
is_synth_data = True
pps = read_files(example_phenopackets_dir)

# convert phenopackets to dataframe
Necessary fields:
- id
- zygosity
- cHGVS mutation
- Obsesity Class and timestamp

In [4]:
from typing import Tuple, Any
from phenopackets import Phenopacket

sex_map = {0: 'unknown', 1: 'female', 2: 'male', 3: 'other'}

def extract_phenotype(phenotypic_features, i) -> Tuple[str, str, bool, int]:
    phenotypic_features = phenotypic_features[i]
    obesity_class_hpo = phenotypic_features.type.id
    obesity_class = phenotypic_features.type.label
    excluded = phenotypic_features.excluded
    onset = phenotypic_features.onset.timestamp.ToSeconds()
    return obesity_class_hpo, obesity_class, excluded, onset

def extract_fields(phenopacket: Phenopacket):
    pp_id = int(phenopacket.id)
    dob = phenopacket.subject.date_of_birth.ToSeconds()
    sex = sex_map[phenopacket.subject.sex]

    try:
        variation_descriptor = phenopacket.interpretations[0].diagnosis.genomic_interpretations[0].variant_interpretation.variation_descriptor
        c_hgvs = variation_descriptor.expressions[1].value
        zygosity = variation_descriptor.allelic_state.label
    except IndexError:
        return None

    phenotypic_features = phenopacket.phenotypic_features

    phenotype_data = []
    for i in range(5):
        try:
            obesity_class_hpo, obesity_class, excluded, onset = extract_phenotype(phenotypic_features, i)
        except IndexError:
            obesity_class_hpo, obesity_class, excluded, onset = None, None, None, None
        phenotype_data.extend([obesity_class_hpo, obesity_class, excluded, onset])

    return tuple([pp_id, dob, sex, zygosity, c_hgvs] + phenotype_data)


In [5]:
# todo: some people have multiple variants max 3

In [6]:
phenopacket = pps[0]
variation_descriptor = phenopacket.interpretations[0].diagnosis.genomic_interpretations[0].variant_interpretation

phenopacket

id: "0"
subject {
  id: "0"
  date_of_birth {
    seconds: 946684800
  }
  sex: FEMALE
  taxonomy {
    id: "NCBITaxon:9606"
    label: "Homo sapiens"
  }
}
phenotypic_features {
  type {
    id: "HP:0001513"
    label: "Obesity"
  }
  excluded: true
  onset {
    timestamp {
      seconds: 708652800
    }
  }
}
interpretations {
  id: "e0dd47b3-6e13-4507-84a7-5606bee17024"
  progress_status: SOLVED
  diagnosis {
    disease {
      id: "ORPHA:71529"
      label: "Obesity due to melanocortin 4 receptor deficiency"
    }
    genomic_interpretations {
      subject_or_biosample_id: "0"
      interpretation_status: CONTRIBUTORY
      variant_interpretation {
        variation_descriptor {
          id: "id:A"
          expressions {
            syntax: "hgvs"
            value: "NP_005903.2:p.(Val103Ile)"
          }
          expressions {
            syntax: "hgvs"
            value: "NM_005912.3:c.181G>T"
          }
          allelic_state {
            id: "GENO:0000135"
            

In [7]:
for pp in pps:
    print(extract_fields(pp))

(0, 946684800, 'female', 'heterozygous', 'NM_005912.3:c.181G>T', 'HP:0001513', 'Obesity', True, 708652800, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(1, 915148800, 'female', 'heterozygous', 'NM_005912.3:c.307G>A', 'HP:0001513', 'Obesity', True, 1254268800, 'HP:0001513', 'Obesity', False, 967507200, None, None, None, None, None, None, None, None, None, None, None, None)
None
None
(12, 757382400, 'male', 'heterozygous', 'NM_005912.3:c.307G>A', 'HP:0001513', 'Obesity', False, 1118793600, 'HP:0025500', 'Class II obesity', False, 1267056000, None, None, None, None, None, None, None, None, None, None, None, None)
(13, 757382400, 'male', 'heterozygous', 'NM_005912.3:c.307G>A', 'HP:0001513', 'Obesity', True, 1234051200, 'HP:0001507', 'Growth abnormality', True, 985219200, 'HP:0001507', 'Growth abnormality', False, 876528000, None, None, None, None, None, None, None, None)
None
None
None
(17, 1072915200, 'female', 'homozygous', 'NM_005912.3:

In [8]:
import polars as pl

data = list(filter(lambda x: x is not None, map(extract_fields, pps)))

data, len(data)

([(0,
   946684800,
   'female',
   'heterozygous',
   'NM_005912.3:c.181G>T',
   'HP:0001513',
   'Obesity',
   True,
   708652800,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None),
  (1,
   915148800,
   'female',
   'heterozygous',
   'NM_005912.3:c.307G>A',
   'HP:0001513',
   'Obesity',
   True,
   1254268800,
   'HP:0001513',
   'Obesity',
   False,
   967507200,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None),
  (12,
   757382400,
   'male',
   'heterozygous',
   'NM_005912.3:c.307G>A',
   'HP:0001513',
   'Obesity',
   False,
   1118793600,
   'HP:0025500',
   'Class II obesity',
   False,
   1267056000,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None),
  (13,
   757382400,
   'male',
   'heterozygous',
   'NM_005912.3:c.307G>A',
   'HP:0001513',
   'Obesity',
   True

In [9]:
columns = ['id', 'date_of_birth', 'sex', 'zygosity', 'c_hgvs', 'obesity_class_hpo0', 'obesity_class0', 'phenotype_refuted0', 'onset0', 'obesity_class_hpo1', 'obesity_class1', 'phenotype_refuted1', 'onset1', 'obesity_class_hpo2', 'obesity_class2', 'phenotype_refuted2', 'onset2', 'obesity_class_hpo3', 'obesity_class3', 'phenotype_refuted3', 'onset3', 'obesity_class_hpo4', 'obesity_class4', 'phenotype_refuted4', 'onset4']
transposed_data = list(zip(*data))

df = pl.DataFrame({col: transposed_data[i] for i, col in enumerate(columns)})

In [10]:
df.head(50)

id,date_of_birth,sex,zygosity,c_hgvs,obesity_class_hpo0,obesity_class0,phenotype_refuted0,onset0,obesity_class_hpo1,obesity_class1,phenotype_refuted1,onset1,obesity_class_hpo2,obesity_class2,phenotype_refuted2,onset2,obesity_class_hpo3,obesity_class3,phenotype_refuted3,onset3,obesity_class_hpo4,obesity_class4,phenotype_refuted4,onset4
i64,i64,str,str,str,str,str,bool,i64,str,str,bool,i64,str,str,bool,i64,str,str,bool,i64,str,str,bool,i64
0,946684800,"""female""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",true,708652800,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,915148800,"""female""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",true,1254268800,"""HP:0001513""","""Obesity""",false,967507200,null,null,null,null,null,null,null,null,null,null,null,null
12,757382400,"""male""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",false,1118793600,"""HP:0025500""","""Class II obesi…",false,1267056000,null,null,null,null,null,null,null,null,null,null,null,null
13,757382400,"""male""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",true,1234051200,"""HP:0001507""","""Growth abnorma…",true,985219200,"""HP:0001507""","""Growth abnorma…",false,876528000,null,null,null,null,null,null,null,null
17,1072915200,"""female""","""homozygous""","""NM_005912.3:c.…","""HP:0025500""","""Class II obesi…",true,638409600,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
18,1136073600,"""female""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",true,916272000,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,1009843200,"""female""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",true,1087257600,"""HP:0001513""","""Obesity""",true,825811200,"""HP:0001507""","""Growth abnorma…",false,976406400,null,null,null,null,null,null,null,null
20,1072915200,"""male""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",true,872208000,"""HP:0025499""","""Class I obesit…",false,674524800,"""HP:0001507""","""Growth abnorma…",false,1037750400,null,null,null,null,null,null,null,null
21,1136073600,"""male""","""heterozygous""","""NM_005912.3:c.…","""HP:0001513""","""Obesity""",true,938563200,"""HP:0001513""","""Obesity""",true,1149724800,"""HP:0001507""","""Growth abnorma…",false,1451606400,null,null,null,null,null,null,null,null


# Data Cleaning

There is only one person with an unspecified zygosity, so we can drop this person.

In [11]:
df = df.filter(df['zygosity'] != 'unspecified zygosity')

In [12]:
# add new timestamps to make the data more realistic
if is_synth_data:
    from ERKER2Phenopackets.src.utils.dateutils import date_to_seconds, seconds_to_date, generate_random_date
    def random_onsets():
        seconds_per_day = 60 * 60 * 24
        dob = generate_random_date('1990-01-01', '2010-01-01')
        dob_seconds = date_to_seconds(dob)
        # dates:
        # 1. untersuchung bei geburt
        # 2. u6: 10 - 12 monate
        # 3. u7: 21 - 24 monate
        # 4. u8: 46 - 48 monate
        # 5. u9: 60 - 64 monate
        intervals = [
            (0, 35),
            (304, 365),
            (639, 730),
            (1399, 1460),
            (1825, 1947)
        ]
    
        onset_dates = [dob_seconds]
        for interval in intervals:
            min_days, max_days = interval
            min_date = seconds_to_date(dob_seconds + min_days * seconds_per_day)
            max_date = seconds_to_date(dob_seconds + max_days * seconds_per_day)
            onset_dates.append(date_to_seconds(generate_random_date(min_date, max_date)))
    
        return onset_dates
    
    dob, u1, u6, u7, u8, u9 = [], [], [], [], [], []
    
    for i in range(df.height):
        cur_dob, cur_u1, cur_u6, cur_u7, cur_u8, cur_u9 = random_onsets()
        dob.append(cur_dob)
        u1.append(cur_u1)
        u6.append(cur_u6)
        u7.append(cur_u7)
        u8.append(cur_u8)
        u9.append(cur_u9)
    
    df = df.with_columns(
        pl.Series(name='date_of_birth', values=dob),
        pl.Series(name='onset0', values=u1),
        pl.Series(name='onset1', values=u6),
        pl.Series(name='onset2', values=u7),
        pl.Series(name='onset3', values=u8),
        pl.Series(name='onset4', values=u9),
    )

ImportError: cannot import name 'seconds_to_date' from 'ERKER2Phenopackets.src.utils.dateutils' (C:\Users\Surface\OneDrive\Documents\DataSpell\ERKER2Phenopackets\ERKER2Phenopackets\src\utils\dateutils.py)

In [ ]:
df.head(50)

# Preprocessing

Order dataframe by descending number of same c_hgvs mutations

In [ ]:
from ERKER2Phenopackets.src.utils.polars_utils import sort_by_method

def c_hgvs_occurences(df: pl.DataFrame) -> pl.DataFrame:
    return  df.join(other=df.groupby('c_hgvs').agg(pl.count('c_hgvs').alias('count')).select(['c_hgvs','count']), on='c_hgvs', how='inner').select(['count'])

df = sort_by_method(df, c_hgvs_occurences)

In [ ]:
# correct value of refuted columns to match phenotype: only Obesity has refuted true these are our negatives
if is_synth_data:
    df = df.with_columns(
        phenotype_refuted0=df['obesity_class0'] == 'Obesity',
        phenotype_refuted1=df['obesity_class1'] == 'Obesity',
        phenotype_refuted2=df['obesity_class2'] == 'Obesity',
        phenotype_refuted3=df['obesity_class3'] == 'Obesity',
        phenotype_refuted4=df['obesity_class4'] == 'Obesity',
    )

In [ ]:
from ERKER2Phenopackets.src.utils.polars_utils import replace_value

# growth abnormality: not recorded phenotype -> replace null
growth_abnormality = 'Growth abnormality'
growth_abnormality_hpo = 'HP:0001507'

for i in range(5):
    df = replace_value(df, f'obesity_class{i}', growth_abnormality, None)
    df = replace_value(df, f'obesity_class_hpo{i}', growth_abnormality_hpo, None)

In [ ]:
from ERKER2Phenopackets.src.analysis.ml.analysis_helper_methods import create_label
from ERKER2Phenopackets.src.utils.polars_utils import map_col

# 0: Obesity refuted
# 1: Overweight
# 2: Class I Obesity (HPO)
# 3: Class II Obesity (HPO)
# 4: Class III Obesity (HPO)

for i in range(5):
    df = map_col(
        df, 
        map_from=[f'obesity_class_hpo{i}', f'obesity_class{i}', f'phenotype_refuted{i}'], 
        map_to=f'obesity_label{i}', 
        mapping=create_label
    )

In [ ]:
# normalize timestamps
df = df.with_columns(
    onset0=df['onset0'] - df['date_of_birth'],
    onset1=df['onset1'] - df['date_of_birth'],
    onset2=df['onset2'] - df['date_of_birth'],
    onset3=df['onset3'] - df['date_of_birth'],
    onset4=df['onset4'] - df['date_of_birth'],
)

In [ ]:
df.head(50)

Overweight, Class I-III Obesity: excluded false
if excluded (refuted) true -> BMI in normal range

only Obesity has refuted true these are our negatives

growth abnormality: not recorded phenotype -> drop rows

# Questions
## Descriptive Statistics
1. Distribution of zygosity? 
2. Distribution of sex?
3. Distribution of c_hgvs mutations?
4. c_hgvs distribution by zygosity?
5. Obesity Class Distribution?
6. Obesity Class Distribution by zygosity?
7. Obesity Class Distribution by c_hgvs mutation?
8. Obesity Class Distribution c_hgvs mutation and zygosity?
9. Obesity Class Distribution by sex?
10. How long does it take for a person to develop the different obesity classes based on their mutation?

## Predictive Statistics
1. What is the probability of a person with a certain c_hgvs mutation to develop a certain obesity class?
2. Prediction model of obesity class based on c_hgvs mutation and zygosity

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ERKER2Phenopackets.src.utils.polars_utils import barchart, barchart_3d, barchart_subplot, barchart_relative_distribution, barchart_relative_distribution_subplot, piechart_subplot, piechart

figsize = None

# Descriptive Statistics
# 1. Zygosity Distribution

In [ ]:
grouped_by_zygosity = df.groupby('zygosity').count()
num_zygosity = grouped_by_zygosity.height

barchart(grouped_by_zygosity['zygosity'], grouped_by_zygosity['count'], figsize=figsize)
piechart(grouped_by_zygosity['zygosity'], grouped_by_zygosity['count'], figsize=figsize)

grouped_by_zygosity.head()

# 2. Sex Distribution

In [ ]:
grouped_by_sex = df.groupby('sex').count()

barchart(grouped_by_sex['sex'], grouped_by_sex['count'], figsize=figsize)
piechart(grouped_by_sex['sex'], grouped_by_sex['count'], figsize=figsize)

grouped_by_sex.head()

# 3. c_hgvs variant distribution

In [ ]:
grouped_by_c_hgvs = df.groupby('c_hgvs').count().sort('count', descending=True)
num_c_hgvs = grouped_by_c_hgvs.height

# barchart(grouped_by_c_hgvs['c_hgvs'], grouped_by_c_hgvs['count'], figsize=figsize, x_tick_rotation='vertical')

barchart_relative_distribution(grouped_by_c_hgvs['c_hgvs'], grouped_by_c_hgvs['count'], figsize=figsize, x_tick_rotation='vertical')

grouped_by_c_hgvs.head(50)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
piechart_subplot(ax1, grouped_by_zygosity['zygosity'], grouped_by_zygosity['count'], 'Zygosity distribution',)
barchart_relative_distribution_subplot(ax2, grouped_by_c_hgvs['c_hgvs'], grouped_by_c_hgvs['count'], 'c_hgvs distribution', x_tick_rotation='vertical')
plt.show()

# 4. c_hgvs distribution by zygosity

In [ ]:
from ERKER2Phenopackets.src.utils.polars_utils import contingency_table

# barchart_3d(df, 'zygosity', 'c_hgvs', figsize=figsize, grouped_by_col1=grouped_by_zygosity, grouped_by_col2=grouped_by_c_hgvs) # not really useful

ct_zygosity_c_hgvs = contingency_table(df, 'zygosity', 'c_hgvs', grouped_by_zygosity['zygosity'], grouped_by_c_hgvs['c_hgvs'])
heterozygous_c_hgvs_frquency, homozygous_c_hgvs_frquency = ct_zygosity_c_hgvs[0], ct_zygosity_c_hgvs[1]
y_lim = int(np.max(ct_zygosity_c_hgvs) * 1.1) + 1

fig, (ax1, ax2) = plt.subplots(1, 2)
barchart_subplot(ax1, grouped_by_c_hgvs['c_hgvs'], heterozygous_c_hgvs_frquency, 'Heterozygous c hgvs distribution', x_tick_rotation='vertical', y_min=0, y_max=y_lim)
barchart_subplot(ax2, grouped_by_c_hgvs['c_hgvs'], homozygous_c_hgvs_frquency, 'Homozygous c hgvs distribution', x_tick_rotation='vertical', y_min=0, y_max=y_lim)
plt.show()

In [ ]:
ct_sex_c_hgvs = contingency_table(df, 'sex', 'c_hgvs', ('male', 'female'), grouped_by_c_hgvs['c_hgvs'])
male_c_hgvs_frquency, female_c_hgvs_frquency = ct_sex_c_hgvs[0], ct_sex_c_hgvs[1]
y_lim = int(np.max(ct_sex_c_hgvs) * 1.1) + 1

fig, (ax1, ax2) = plt.subplots(1, 2)
barchart_subplot(ax1, grouped_by_c_hgvs['c_hgvs'], male_c_hgvs_frquency, 'Male c hgvs distribution', x_tick_rotation='vertical', y_min=0, y_max=y_lim)
barchart_subplot(ax2, grouped_by_c_hgvs['c_hgvs'], female_c_hgvs_frquency, 'Female c hgvs distribution', x_tick_rotation='vertical', y_min=0, y_max=y_lim)
plt.show()

# 4. Obesity Class Distribution

In [ ]:
from ERKER2Phenopackets.src.utils.polars_utils import melt_groupby_count, sort_columns

# 4.1 count appearance of obesity classes/ hpo terms at each of the 3 inspections
obesity_class_count = melt_groupby_count(df, columns=[f'obesity_label{i}' for i in range(5)]).sort(by='value')

obesity_class_count = sort_columns(obesity_class_count, ['value'])

print(obesity_class_count.head(50))

In [ ]:
column_names = tuple(obesity_class_count['value'])
obesity_class_count_transpose = obesity_class_count.transpose(column_names=column_names)
if column_names != obesity_class_count_transpose.row(0):
    raise Exception('New column names do not match old row names')
obesity_class_count_transpose = obesity_class_count_transpose.tail(5).select(['Refuted Obesity (HP:0001513)', 'Overweight (HP:0025502)', 'Class I obesity (HP:0025499)', 'Class II obesity (HP:0025500)', 'Class III obesity (HP:0025501)'])
print(obesity_class_count_transpose.head(50))

In [ ]:
def cast(column):
    return column.cast(pl.Int32).to_list()

In [ ]:
categories = obesity_class_count_transpose.columns

time_stamps = ['U1', 'U6', 'U7', 'U8', 'U9']


values = [cast(obesity_class_count_transpose[cat]) for cat in categories]

plt.figure(figsize=figsize)

plt.stackplot(range(5), values[0], values[1], values[2], values[3], values[4], labels=categories)
plt.xticks(range(5), time_stamps)
plt.ylabel('Cases')
plt.legend()
plt.show()

In [ ]:
from ERKER2Phenopackets.src.utils.polars_utils import barchart_multiple

barchart_multiple(time_stamps, values, 'Cases', categories)

# 4.1 Phenotype Transition Graphs
1. Markov chain style

In [ ]:
from ERKER2Phenopackets.src.analysis.ml.analysis_helper_methods import plot_phenotype_transition_graph

file_path = 'markov_chain_obesity_classes.png'
phenotype_labels = ['Refuted Obesity \n(HP:0001513)', 'Overweight \n(HP:0025502)', 'Class I obesity \n(HP:0025499)', 'Class II obesity \n(HP:0025500)', 'Class III obesity \n(HP:0025501)']

initial_counts_df = (melt_groupby_count(df, columns=
[f'obesity_label{i}' for i in range(5)])
                     .select(['value', 'obesity_label0'])
                     )
initial_counts_dict = {
    row['value']: row['obesity_label0']
    for row in initial_counts_df.to_struct('test')
}
initial_counts = (
        [initial_counts_dict[pheno_label]
         for pheno_label in categories]
        + [initial_counts_dict['Not recorded']]
)

plot_phenotype_transition_graph(df, 'obesity_label', phenotype_labels.copy(), initial_counts=initial_counts, count_not_recorded=True, percentages=False, file_path=file_path)

2. Patient Journey

In [ ]:
from ERKER2Phenopackets.src.analysis.ml.analysis_helper_methods import index_similar
from ERKER2Phenopackets.src.utils.graphutils import graphplot
import networkx as nx

# def connect_rows(num_rows, num_nodes_per_row):
#     for row in range(1, num_rows):
#         for node_in_first_row in range(1, num_nodes_per_row + 1):
#             for node_in_next_row in range(1, num_nodes_per_row + 1):
#                 current_node_index = (row - 1) * num_nodes_per_row + node_in_first_row
#                 next_node_index = row * num_nodes_per_row + node_in_next_row
#                 yield current_node_index, next_node_index

file_path = 'time_split_graph_obesity_classes.png'
num_phenotype_labels = len(phenotype_labels)
num_phenotype_measurements = 5
source = 0
sink = len(categories)*5+1
not_recorded = -1

G = nx.DiGraph()
G.add_node(source, label='Source')
G.add_node(sink, label='Sink')
G.add_node(not_recorded, label='Not recorded')

node = 1
empty_nodes = []
# transition_counts = {i: [] for i in range(sink)}
for i in range(num_phenotype_measurements):
    target_addend = 1 + (i + 1) * num_phenotype_labels
    
    for label in phenotype_labels:
        row, col = i, (node - 1) % num_phenotype_labels
        count = int(obesity_class_count_transpose.item(row, col))
        if count == 0:
            empty_nodes.append(node)
        else:
            G.add_node(node, label=f'{label}\n{time_stamps[i]}\n{count}/{df.height}')
            
            if i < num_phenotype_measurements - 1:
                label_wo_newlines = categories[index_similar(categories, label)]
                # target nodes and how many patients transition to these targets
                outgoing = df.filter(df[f'obesity_label{i}'] == label_wo_newlines).select([f'obesity_label{i + 1}']).groupby(by=[f'obesity_label{i + 1}']).count()
                
                
                for target_label, count_transitioning_to_target in outgoing.rows():
                    target = not_recorded
                    try:
                        target = categories.index(target_label)
                    except ValueError:
                        pass
                    else: # if label in row
                        target += target_addend
                        
                    # add the edge
                    if G.has_edge(node, target):
                        G.edges[node, target]['denominator'] += count
                        G.edges[node, target]['numerator'] += count_transitioning_to_target
                    else:
                        G.add_edge(node, target, denominator=count, numerator=count_transitioning_to_target)   
                
        node += 1

# connect first row to source
for i in range(1, num_phenotype_labels + 1):
    if not i in empty_nodes:
        G.add_edge(source, i)

for edge in G.edges(data=True):
    from_, to, data = edge
    if data:
        G.edges[from_, to]['label'] = f'{data["numerator"]}/{data["denominator"]}'

# connect last row to sink 
for i in range(len(categories)*4+1, len(categories)*5+1):
    if not i in empty_nodes:
        G.add_edge(i, sink)
G.remove_node(not_recorded)

ccs = list(nx.connected_components(G.to_undirected()))
# determine largest connected component (this is the main graph) and remove it
max_component_id, max_component_length = max(
    ((i, len(component)) for i, component in enumerate(ccs)),
    key=lambda x: x[1],
    default=(-1, 0)
)
ccs = [component for i, component in enumerate(ccs) if i != max_component_id]

for cc in ccs: # assume each cc could have multiple nodes in multiple layers
    cc = list(cc)
    # separate the sinks and sources of the cc into layers 
    sources, sinks = [node for node in cc if G.in_degree(node) == 0], [node for node in cc if G.out_degree(node) == 0]
    layers = {i: [node for node in cc if i * num_phenotype_labels <= node < (i + 1) * num_phenotype_labels] for i in range(num_phenotype_measurements)}

    # 1. 
    while len(sources) > 0:
        for node in sources:
            previous = node - num_phenotype_labels
            if previous < source:
                previous = source

            if not G.has_node(previous):
                G.add_node(previous, label=f'{phenotype_labels[(previous - 1) % num_phenotype_labels]}\n{time_stamps[(previous - 1) // num_phenotype_labels]}\n?/?') # todo

            if not G.has_edge(previous, node):
                G.add_edge(previous, node, label="?") # todo

            if previous != source:
                cc.append(previous)

        sources = [node for node in cc if G.in_degree(node) == 0]


    # 2. find the last layer, go forwards until you hit source and add the connections
    while len(sinks) > 0:
        # 1. 
        for node in sinks:
            next_ = node + num_phenotype_labels
            if next_ > sink:
                next_ = sink

            if not G.has_node(next_):
                G.add_node(next_, label=f'{phenotype_labels[(next_ - 1) % num_phenotype_labels]}\n{time_stamps[(next_ - 1) // num_phenotype_labels]}\n?/?') # todo

            if not G.has_edge(node, next_):
                G.add_edge(node, next_, label="?") # todo

            if next_ != sink:
                cc.append(next_)

        sinks = [node for node in cc if G.out_degree(node) == 0]

    # todo: add counts to the edges and nodes that were filled in

graphplot(G, file_path=file_path, orientation="LR")

# 4.2 Table Obesity Class distribution in general and how long it takes to develop each class on average

In [ ]:
from typing import List, Union
import polars as pl

def time_to_phenotype(df, phenotype_col_prefix, timestamp_col_prefix, time_unit_df: str, time_unit_output: str, by_col_labels: Union[str,List[str]] = None):
    """
    
    Expects a polars dataframe in ERKER format, with 5 label columns and 5 timestamp columns. For each patient, it notes when the patient was first diagnosed with a label and then calculates the average time it took for the patient to be diagnosed with the label. This data is then averaged in a new dataframe to show the average time it takes for a patient to be diagnosed with a label. 
    :param df: 
    :param phenotype_col_prefix: 
    :param timestamp_col_prefix:
    :param by_col_labels: columns to stratify by
    :return: 
    """
    if by_col_labels and isinstance(by_col_labels, str): # wrap in list
        by_col_labels = [by_col_labels]
        
    label_cols = [col for col in df.columns if col.startswith(phenotype_col_prefix)]
    timestamp_cols = [col for col in df.columns if col.startswith(timestamp_col_prefix)]

    new_data = {'id': [], 'phenotype': [], 'timestamp': []}
    
    if by_col_labels:
        for by_col_label in by_col_labels:
            new_data[by_col_label] = []
        for id_, by_cols, labels, timestamps in zip(df['id'], zip(*[df[by_col_label] for by_col_label in by_col_labels]), zip(*[df[col] for col in label_cols]), zip(*[df[col] for col in timestamp_cols])):
            seen_labels = set()
            for label, timestamp in zip(labels, timestamps):
                if label not in seen_labels:
                    new_data['id'].append(id_)
                    for i, by_col_label in enumerate(by_col_labels):
                        new_data[by_col_label].append(by_cols[i])
                    new_data['phenotype'].append(label)
                    new_data['timestamp'].append(timestamp)
                    seen_labels.add(label)
        groupby = by_col_labels + ['phenotype']
    else:
        for id_, labels, timestamps in zip(df['id'], zip(*[df[col] for col in label_cols]),
                                           zip(*[df[col] for col in timestamp_cols])):
            seen_labels = set()
            for label, timestamp in zip(labels, timestamps):
                if label not in seen_labels:
                    new_data['id'].append(id_)
                    new_data['phenotype'].append(label)
                    new_data['timestamp'].append(timestamp)
                    seen_labels.add(label)
        groupby = ['phenotype']

    result =  pl.DataFrame(new_data).groupby(groupby).agg(
        pl.mean('timestamp').alias('avg_timestamp'),
        pl.std('timestamp').alias('std_timestamp'),
        pl.count('timestamp').alias('count')
    )
    
    # change time units
    unit_switch = {
        'seconds': 1,
        'minutes': 60,
        'hours': 60 * 60,
        'days': 60 * 60 * 24,
        'months': 60 * 60 * 24 * 30,
        'years': 60 * 60 * 24 * 365
    }
    change_unit = unit_switch[time_unit_df] / unit_switch[time_unit_output]
    result = result.with_columns(
        (pl.col('avg_timestamp') * change_unit).apply(lambda x: round(x, 2)).alias('avg_timestamp_' + time_unit_output),
        (pl.col('std_timestamp') * change_unit).apply(lambda x: round(x, 2)).alias('std_timestamp_' + time_unit_output),
    ).drop('avg_timestamp', 'std_timestamp').sort(by='avg_timestamp_' + time_unit_output)
    
    # add total count and percentage
    total_counts = result.groupby(by_col_labels).agg(pl.sum('count').alias('total_count'))
    result = result.join(total_counts, on=by_col_labels, how='left').select(
        [col for col in result.columns if col != 'count' and col != 'total_count'] + ['count', 'total_count']
    )
    
    result = result.with_columns((pl.col('count') / pl.col('total_count')).alias('frequency'))
    
    return result
output_unit = 'months'
avg_time_phenotype = time_to_phenotype(df, 'obesity_label', 'onset', time_unit_df='seconds', time_unit_output=output_unit, by_col_labels=['c_hgvs', 'zygosity'])
avg_time_phenotype = avg_time_phenotype.filter(pl.col('phenotype') != 'Not recorded').sort(by='avg_timestamp_' + output_unit)

# todo: p values for difference in avg time to phenotype between categories

In [ ]:
avg_time_phenotype

In [ ]:
from ERKER2Phenopackets.src.utils.polars_utils import scatter_plot, scatter_subplot
# todo: maybe change this to include ranges (std) and not just the mean
avg_time_phenotype = avg_time_phenotype.sort(by='phenotype', descending=False)
x = avg_time_phenotype['avg_timestamp_months']
y_labels = ["Refuted Obesity (HP:0001513)", "Overweight (HP:0025502)", "Class I obesity (HP:0025499)", "Class II obesity (HP:0025500)", "Class III obesity (HP:0025501)"]
phenotyp_to_number = {phenotype: i for i, phenotype in enumerate(y_labels)}
y = np.array([phenotyp_to_number[phenotype] for phenotype in avg_time_phenotype['phenotype']])
z = avg_time_phenotype['c_hgvs']

fig, ax = plt.subplots(1, 1, figsize=(15,10))
ax.axhspan(ymin=3.75, ymax=4.25, xmin=0.05, xmax=0.89, color='red', alpha=0.3, hatch='//')
scatter_subplot(ax, x, y, z, title='Average time to phenotype', z_discrete=True, y_labels=y_labels, x_label='Months')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = [np.random.normal(0, std, 100) for std in range(1, 4)]
plt.violinplot(data, vert=False)
plt.show()

# Which combinations lead to the development of Class III obesity (HP:0025501) before 5 years of age (60 months)?

In [ ]:
before_5 = avg_time_phenotype.filter((pl.col('phenotype') == "Class III obesity (HP:0025501)") & (pl.col('avg_timestamp_months') < 59))
if before_5.is_empty():
    print('No combinations lead to the development of Class III obesity (HP:0025501) before 5 years of age (60 months)')
else:
    before_5

# Which combinations lead to a high risk of developing Class III obesity (HP:0025501)?

In [ ]:
risk_factor = 0.1
high_risk = avg_time_phenotype.filter((pl.col('phenotype') == "Class III obesity (HP:0025501)") & (pl.col('frequency') > risk_factor))
if high_risk.is_empty():
    print('No combinations lead to a high risk of developing Class III obesity (HP:0025501)')
else:
    print(high_risk)

In [ ]:
# todo: std is null for real data for some reason and some patients are counte double in the count column

# Classification

## prepare dataset for classification

In [ ]:
df

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

# Load the Iris dataset (or any dataset of your choice)
data = load_iris()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize models for different classifiers
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy:", accuracy)